# Filtado de mensajes spam

## Descripción del problema real

La recepción de publicidad no deseada a traves mensajes de texto usando SMS (Short Message Service) es un problema que afecta a muchos usuarios de teléfonos móviles. El problema radica en que los usuarios deben pagar por los mesajes recibidos, y por este motivo resulta muy importante que las compañías prestadoras del servicio puedan filtrar mensajes indeseados antes de enviarlos a su destinatario final. Los mensajes tienen una longitud máxima de 160 caracteres, por lo que el texto resulta poco para realizar la clasificación, en comparación con textos más largos (como los emails). Adicionalmente, los errores de digitación dificultan el proceso de detección automática.

## Descripción del problema en términos de los datos

Se tiene una muestra contiene 5574 mensajes en inglés, no codificados y clasificados como legítimos (ham) o spam (http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). La información está almacenada en el archivo `datos/spam-sms.zip`.El problema en términos de los datos consiste en clasificar si un mensaje SMS es legítico o spam, a partir del análisis de las palabras que contiente, partiendo del supuesto de que ciertas palabras que son más frecuentes dependiendo del tipo de mensaje. Esto implica que en la fase de preparación de los datos se deben extraer las palabras que contiene cada mensaje para poder realizar el análsis.

## Aproximaciones posibles

En este caso, se desea comparar los resultados de un modelo de redes neuronales artificiales y otras técnicas estadísticas para realizar la clasificación.

## Requerimientos

Usted debe:

* Preprocesar los datos para representarlos usando bag-of-words.


* Construir un modelo de regresión logística como punto base para la comparación con otros modelos más complejos.


* Construir un modelo de redes neuronales artificiales. Asimismo, debe determinar el número de neuronas en la capa o capas ocultas.


* Utiizar una técnica como crossvalidation u otra similar para establecer la robustez del modelo.


* Presentar métricas de desempeño para establecer las bondades y falencias de cada clasificador.

### Imports

In [1]:
import re 
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix as confusion
from sklearn.metrics import accuracy_score as precision
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import KFold as KF
from tensorflow import keras
import tensorflow as tf


C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\HP\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1)

## Lectura de los datos

In [2]:
datos = pd.DataFrame(columns=['etiqueta','contenido'])

def limpieza(txt):
    return re.sub("\s{1,}"," ",re.sub('[0-9]*', '', re.sub('https?:\/\/.*[\n]*','', txt))).replace('&gt;','').replace('&lt;','').translate(str.maketrans('', '', string.punctuation))

with open('datos/SMSSpamCollection.txt', encoding='utf-8') as abierto:
    lista = str(abierto.read()).split("\n")
    for row in lista:
        patron = "ham\s"
        remplazo = "1,,. "
        row = re.sub(patron, remplazo, row)
        patron = "spam\s"
        remplazo = "0,,. "
        row = re.sub(patron, remplazo, row)
        row = row.split(",,.")
        if(len(row)>1):
            datos = datos.append({
                'etiqueta': row[0],
                'contenido': limpieza(row[1]) 
                }, ignore_index=True) 

### Eliminación de palabras no significativas y creción de la bolsa de palabras

In [3]:
X = datos['contenido']
Y = datos['etiqueta'].astype('int')

stem = nltk.stem.SnowballStemmer('english')

## palabras propias del lenguaje sin aporte al análiss
propias = stopwords.words("english")
def aplicarStem(txt):   
   
    words = [stem.stem(word) for word in txt.split() if word.lower() not in propias]
    return " ".join(words)

ss = []
for i in X:
    ss.append(aplicarStem(i))

# Bolsa de palabras

vectorizador = CountVectorizer(ss)
sX = vectorizador.fit_transform(ss)

sX = sX.toarray()
X_train, X_test, Y_train, Y_test = tts(sX, Y , stratify = Y, test_size = 0.2, random_state=101)


# Regresión Logística

In [30]:
regresion = LogisticRegression()
regresion.fit(X_train, Y_train)
prediccion = regresion.predict(X_test)

print(
      "Matriz de confusión: ", confusion(Y_test, prediccion),
      "\n",
      "Precisión: ",precision(Y_test, prediccion)
)


Matriz de confusión:  [[129  20]
 [  2 964]] 
 Precisión:  0.9802690582959641


# Modelo red neuronal

### Se elige la cantidad de neuronas

In [5]:

nresults = [0,0]
for i in range(1,21):
    ## Crea un modelo vacio    
    model = keras.Sequential()
    ## Adiciona las capas
    model.add(keras.layers.Dense(units = i, activation=tf.nn.relu, input_dim = X_train.shape[1]))
    model.add(keras.layers.Dense(units = 1, activation=tf.nn.sigmoid))
    
    model.summary()
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['acc']
                  )
    ## Estructura del modelo creado
    model.fit(X_train,Y_train,epochs=50,batch_size=30)
    r = model.evaluate(X_test, Y_test)
    if(r[1]>nresults[1]):
        nresults = [i,r[1]]
    model = 0

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 7133      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2         
Total params: 7,135
Trainable params: 7,135
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/50
4459/4459 [==============================] - 1s 150us/sample - loss: 0.6561 - acc: 0.8986
Epoch 2/50
4459/4459 [==============================] - 1s 129us/sample - loss: 0.5721 - acc: 0.9789
Epoch 3/50
4459/4459 [==============================] - 1s 122us/sample - loss: 0.5037 - acc: 0.9872
E

4459/4459 [==============================] - 1s 125us/sample - loss: 0.0364 - acc: 0.9957
Epoch 12/50
4459/4459 [==============================] - 1s 124us/sample - loss: 0.0318 - acc: 0.9960
Epoch 13/50
4459/4459 [==============================] - 1s 122us/sample - loss: 0.0280 - acc: 0.9962
Epoch 14/50
4459/4459 [==============================] - 1s 122us/sample - loss: 0.0249 - acc: 0.9962
Epoch 15/50
4459/4459 [==============================] - 1s 125us/sample - loss: 0.0223 - acc: 0.9964
Epoch 16/50
4459/4459 [==============================] - 1s 121us/sample - loss: 0.0201 - acc: 0.9966
Epoch 17/50
4459/4459 [==============================] - 1s 128us/sample - loss: 0.0182 - acc: 0.9969
Epoch 18/50
4459/4459 [==============================] - 1s 129us/sample - loss: 0.0165 - acc: 0.9971
Epoch 19/50
4459/4459 [==============================] - 1s 128us/sample - loss: 0.0151 - acc: 0.9971
Epoch 20/50
4459/4459 [==============================] - 1s 123us/sample - loss: 0.0138 - acc:

4459/4459 [==============================] - 1s 125us/sample - loss: 0.0456 - acc: 0.9971
Epoch 35/50
4459/4459 [==============================] - 1s 122us/sample - loss: 0.0435 - acc: 0.9971
Epoch 36/50
4459/4459 [==============================] - 1s 124us/sample - loss: 0.0415 - acc: 0.9971
Epoch 37/50
4459/4459 [==============================] - 1s 121us/sample - loss: 0.0397 - acc: 0.9971
Epoch 38/50
4459/4459 [==============================] - 1s 124us/sample - loss: 0.0381 - acc: 0.9971
Epoch 39/50
4459/4459 [==============================] - 1s 122us/sample - loss: 0.0366 - acc: 0.9971
Epoch 40/50
4459/4459 [==============================] - 1s 122us/sample - loss: 0.0352 - acc: 0.9971
Epoch 41/50
4459/4459 [==============================] - 1s 123us/sample - loss: 0.0339 - acc: 0.9971
Epoch 42/50
4459/4459 [==============================] - 1s 129us/sample - loss: 0.0327 - acc: 0.9971
Epoch 43/50
4459/4459 [==============================] - 1s 133us/sample - loss: 0.0315 - acc:

Epoch 1/50
4459/4459 [==============================] - 1s 191us/sample - loss: 0.4946 - acc: 0.8870
Epoch 2/50
4459/4459 [==============================] - 1s 179us/sample - loss: 0.2517 - acc: 0.9507
Epoch 3/50
4459/4459 [==============================] - 1s 185us/sample - loss: 0.1506 - acc: 0.9731
Epoch 4/50
4459/4459 [==============================] - 1s 182us/sample - loss: 0.0979 - acc: 0.9843
Epoch 5/50
4459/4459 [==============================] - 1s 178us/sample - loss: 0.0687 - acc: 0.9886
Epoch 6/50
4459/4459 [==============================] - 1s 177us/sample - loss: 0.0513 - acc: 0.9917
Epoch 7/50
4459/4459 [==============================] - 1s 175us/sample - loss: 0.0398 - acc: 0.9935
Epoch 8/50
4459/4459 [==============================] - 1s 170us/sample - loss: 0.0320 - acc: 0.9953
Epoch 9/50
4459/4459 [==============================] - 1s 151us/sample - loss: 0.0263 - acc: 0.9960
Epoch 10/50
4459/4459 [==============================] - 1s 133us/sample - loss: 0.0220 - a

4459/4459 [==============================] - 1s 124us/sample - loss: 0.0051 - acc: 0.9987
Epoch 25/50
4459/4459 [==============================] - 1s 124us/sample - loss: 0.0048 - acc: 0.9987
Epoch 26/50
4459/4459 [==============================] - 1s 127us/sample - loss: 0.0045 - acc: 0.9987
Epoch 27/50
4459/4459 [==============================] - 1s 124us/sample - loss: 0.0042 - acc: 0.9987
Epoch 28/50
4459/4459 [==============================] - 1s 128us/sample - loss: 0.0040 - acc: 0.9987
Epoch 29/50
4459/4459 [==============================] - 1s 131us/sample - loss: 0.0038 - acc: 0.9987
Epoch 30/50
4459/4459 [==============================] - 1s 140us/sample - loss: 0.0036 - acc: 0.9987
Epoch 31/50
4459/4459 [==============================] - 1s 124us/sample - loss: 0.0034 - acc: 0.9987
Epoch 32/50
4459/4459 [==============================] - 1s 121us/sample - loss: 0.0033 - acc: 0.9987
Epoch 33/50
4459/4459 [==============================] - 1s 121us/sample - loss: 0.0031 - acc:

...

4459/4459 [==============================] - 1s 134us/sample - loss: 0.0010 - acc: 0.9991
Epoch 46/50
4459/4459 [==============================] - 1s 135us/sample - loss: 9.7118e-04 - acc: 0.9991
Epoch 47/50
4459/4459 [==============================] - 1s 134us/sample - loss: 9.4140e-04 - acc: 0.9991
Epoch 48/50
4459/4459 [==============================] - 1s 140us/sample - loss: 9.1394e-04 - acc: 0.9991
Epoch 49/50
4459/4459 [==============================] - 1s 134us/sample - loss: 8.8748e-04 - acc: 0.9991
Epoch 50/50
1115/1115 [==============================] - 0s 159us/sample - loss: 0.1197 - acc: 0.9821
Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 15)                106995    
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 16        
Total params: 107,011
Trainable

4459/4459 [==============================] - 1s 149us/sample - loss: 0.0082 - acc: 0.9984
Epoch 12/50
4459/4459 [==============================] - 1s 149us/sample - loss: 0.0070 - acc: 0.9984
Epoch 13/50
4459/4459 [==============================] - 1s 150us/sample - loss: 0.0060 - acc: 0.9989
Epoch 14/50
4459/4459 [==============================] - 1s 149us/sample - loss: 0.0053 - acc: 0.9989
Epoch 15/50
4459/4459 [==============================] - 1s 150us/sample - loss: 0.0046 - acc: 0.9993
Epoch 16/50
4459/4459 [==============================] - 1s 150us/sample - loss: 0.0042 - acc: 0.9993
Epoch 17/50
4459/4459 [==============================] - 1s 148us/sample - loss: 0.0037 - acc: 0.9993
Epoch 18/50
4459/4459 [==============================] - 1s 151us/sample - loss: 0.0034 - acc: 0.9993
Epoch 19/50
4459/4459 [==============================] - 1s 151us/sample - loss: 0.0030 - acc: 0.9993
Epoch 20/50
4459/4459 [==============================] - 1s 148us/sample - loss: 0.0028 - acc:

4459/4459 [==============================] - 1s 139us/sample - loss: 7.1254e-04 - acc: 1.0000
Epoch 34/50
4459/4459 [==============================] - 1s 138us/sample - loss: 6.4714e-04 - acc: 1.0000
Epoch 35/50
4459/4459 [==============================] - 1s 139us/sample - loss: 5.8948e-04 - acc: 1.0000
Epoch 36/50
4459/4459 [==============================] - 1s 139us/sample - loss: 5.2987e-04 - acc: 1.0000
Epoch 37/50
4459/4459 [==============================] - 1s 140us/sample - loss: 4.8416e-04 - acc: 1.0000
Epoch 38/50
4459/4459 [==============================] - 1s 140us/sample - loss: 4.4219e-04 - acc: 1.0000
Epoch 39/50
4459/4459 [==============================] - 1s 137us/sample - loss: 4.0695e-04 - acc: 1.0000
Epoch 40/50
4459/4459 [==============================] - 1s 141us/sample - loss: 3.6723e-04 - acc: 1.0000
Epoch 41/50
4459/4459 [==============================] - 1s 161us/sample - loss: 3.3354e-04 - acc: 1.0000
Epoch 42/50
4459/4459 [==============================] - 1

Epoch 1/50
4459/4459 [==============================] - 1s 232us/sample - loss: 0.4735 - acc: 0.9320
Epoch 2/50
4459/4459 [==============================] - 1s 196us/sample - loss: 0.1673 - acc: 0.9800
Epoch 3/50
4459/4459 [==============================] - 1s 181us/sample - loss: 0.0798 - acc: 0.9892
Epoch 4/50
4459/4459 [==============================] - 1s 183us/sample - loss: 0.0480 - acc: 0.9937
Epoch 5/50
4459/4459 [==============================] - 1s 183us/sample - loss: 0.0329 - acc: 0.9960
Epoch 6/50
4459/4459 [==============================] - 1s 183us/sample - loss: 0.0242 - acc: 0.9960
Epoch 7/50
4459/4459 [==============================] - 1s 183us/sample - loss: 0.0187 - acc: 0.9969
Epoch 8/50
4459/4459 [==============================] - 1s 178us/sample - loss: 0.0148 - acc: 0.9973
Epoch 9/50
4459/4459 [==============================] - 1s 179us/sample - loss: 0.0120 - acc: 0.9975
Epoch 10/50
4459/4459 [==============================] - 1s 195us/sample - loss: 0.0099 - a

4459/4459 [==============================] - 1s 161us/sample - loss: 0.0017 - acc: 0.9993
Epoch 25/50
4459/4459 [==============================] - 1s 158us/sample - loss: 0.0015 - acc: 0.9993
Epoch 26/50
4459/4459 [==============================] - 1s 156us/sample - loss: 0.0014 - acc: 0.9993
Epoch 27/50
4459/4459 [==============================] - 1s 157us/sample - loss: 0.0013 - acc: 0.9993
Epoch 28/50
4459/4459 [==============================] - 1s 168us/sample - loss: 0.0012 - acc: 0.9993
Epoch 29/50
4459/4459 [==============================] - 1s 164us/sample - loss: 0.0011 - acc: 0.9996
Epoch 30/50
4459/4459 [==============================] - 1s 156us/sample - loss: 0.0010 - acc: 0.9996
Epoch 31/50
4459/4459 [==============================] - 1s 155us/sample - loss: 9.4037e-04 - acc: 0.9996
Epoch 32/50
4459/4459 [==============================] - 1s 154us/sample - loss: 8.6622e-04 - acc: 0.9998
Epoch 33/50
4459/4459 [==============================] - 1s 156us/sample - loss: 8.031

### Se elige la cantidad de capas

In [6]:
cresults = [0,0]
for i in range(1,10):
    ## Crea un modelo vacio    
    model = keras.Sequential()
    ## Adiciona las capas
    for j in range(0,i):
        model.add(keras.layers.Dense(units =nresults[0], activation=tf.nn.relu, input_dim = X_train.shape[1]))
    model.add(keras.layers.Dense(units = 1, activation=tf.nn.sigmoid))
    
    model.summary()
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['acc']
                  )
    ## Estructura del modelo creado
    model.fit(X_train,Y_train,epochs=50,batch_size=30)
    r = model.evaluate(X_test, Y_test)
    if(r[1]>cresults[1]):
        cresults = [i,r[1]]
    model = 0

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 1)                 7133      
_________________________________________________________________
dense_41 (Dense)             (None, 1)                 2         
Total params: 7,135
Trainable params: 7,135
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
4459/4459 [==============================] - 1s 172us/sample - loss: 0.6586 - acc: 0.9051
Epoch 2/50
4459/4459 [==============================] - 1s 133us/sample - loss: 0.5808 - acc: 0.9827
Epoch 3/50
4459/4459 [==============================] - 1s 141us/sample - loss: 0.5122 - acc: 0.9861
Epoch 4/50
4459/4459 [==============================] - 1s 142us/sample - loss: 0.4541 - acc: 0.9904
Epoch 5/50
4459/4459 [==============================] - 1s 143us/sample - loss: 0.4050 - acc: 0.9919
Epoch 6/

Epoch 17/50
4459/4459 [==============================] - 1s 166us/sample - loss: 0.1303 - acc: 0.9964
Epoch 18/50
4459/4459 [==============================] - 1s 166us/sample - loss: 0.1209 - acc: 0.9964
Epoch 19/50
4459/4459 [==============================] - 1s 164us/sample - loss: 0.1124 - acc: 0.9964
Epoch 20/50
4459/4459 [==============================] - 1s 166us/sample - loss: 0.1047 - acc: 0.9964
Epoch 21/50
4459/4459 [==============================] - 1s 166us/sample - loss: 0.0978 - acc: 0.9966
Epoch 22/50
4459/4459 [==============================] - 1s 168us/sample - loss: 0.0915 - acc: 0.9966
Epoch 23/50
4459/4459 [==============================] - 1s 166us/sample - loss: 0.0857 - acc: 0.9966
Epoch 24/50
4459/4459 [==============================] - 1s 165us/sample - loss: 0.0804 - acc: 0.9966
Epoch 25/50
4459/4459 [==============================] - 1s 167us/sample - loss: 0.0756 - acc: 0.9966
Epoch 26/50
4459/4459 [==============================] - 1s 165us/sample - loss: 0

## Cantidades óptimas de neuronas y capas

In [12]:
print("Neuronas ---- " ,nresults)
print("Capas ---- " ,cresults)

Neuronas ----  [1, 0.9883408]
Capas ----  [1, 0.9883408]


## Modelo definitivo

In [8]:
model = keras.Sequential()
## Adiciona las capas
for j in range(0,cresults[0]):
    model.add(keras.layers.Dense(units =nresults[0], activation=tf.nn.relu, input_dim = X_train.shape[1]))
model.add(keras.layers.Dense(units = 1, activation=tf.nn.sigmoid))
    
model.summary()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc','mse']
              )
## Estructura del modelo creado
model.fit(X_train,Y_train,epochs=100,batch_size=30)
r = model.evaluate(X_test, Y_test)


Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             (None, 1)                 7133      
_________________________________________________________________
dense_46 (Dense)             (None, 1)                 2         
Total params: 7,135
Trainable params: 7,135
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
4459/4459 [==============================] - 1s 256us/sample - loss: 0.5350 - acc: 0.8643 - mean_squared_error: 0.1746
Epoch 2/100
4459/4459 [==============================] - 1s 195us/sample - loss: 0.3439 - acc: 0.8659 - mean_squared_error: 0.0962
Epoch 3/100
4459/4459 [==============================] - 1s 177us/sample - loss: 0.2641 - acc: 0.8659 - mean_squared_error: 0.0718
Epoch 4/100
4459/4459 [==============================] - 1s 171us/sample - loss: 0.2224 - acc: 0.8659 - mean_squared_err

4459/4459 [==============================] - 1s 174us/sample - loss: 0.0053 - acc: 0.9996 - mean_squared_error: 2.9108e-04
Epoch 58/100
4459/4459 [==============================] - 1s 173us/sample - loss: 0.0050 - acc: 0.9996 - mean_squared_error: 2.7075e-04
Epoch 59/100
4459/4459 [==============================] - 1s 170us/sample - loss: 0.0047 - acc: 1.0000 - mean_squared_error: 2.4377e-04
Epoch 60/100
4459/4459 [==============================] - 1s 177us/sample - loss: 0.0044 - acc: 1.0000 - mean_squared_error: 2.2394e-04
Epoch 61/100
4459/4459 [==============================] - 1s 176us/sample - loss: 0.0042 - acc: 1.0000 - mean_squared_error: 2.0359e-04
Epoch 62/100
4459/4459 [==============================] - 1s 185us/sample - loss: 0.0039 - acc: 1.0000 - mean_squared_error: 1.8791e-04
Epoch 63/100
4459/4459 [==============================] - 1s 193us/sample - loss: 0.0037 - acc: 1.0000 - mean_squared_error: 1.7299e-04
Epoch 64/100
4459/4459 [==============================] - 1s 

In [31]:
print("Precisión del modelo definitivo ", r[1])
print("MSE del modelo definitivo ", r[2])

Precisión del modelo definitivo  0.97847533
MSE del modelo definitivo  0.020242328


## Validación Cruzada

In [26]:
def validacion(indice_entren, indice_test, X, Y):
    
    X_train = X[indice_entren]
    X_test = X[indice_test]
    Y_train = Y.iloc[indice_entren]
    Y_test = Y.iloc[indice_test]
        
    model = keras.Sequential()
    ## Adiciona las capas
    for j in range(0,cresults[0]):
        model.add(keras.layers.Dense(units =nresults[0], activation=tf.nn.relu, input_dim = X_train.shape[1]))
    model.add(keras.layers.Dense(units = 1, activation=tf.nn.sigmoid))
    
    model.summary()
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc','mse']
              )
## Estructura del modelo creado
    model.fit(X_train,Y_train,epochs=100,batch_size=30)
    r = model.evaluate(X_test, Y_test)

            
    return r[2]
    
kf = KF(n_splits=4)

crossval = []
for indice_entren, indice_test in kf.split(X_train):
    crossval.append(validacion(indice_entren,indice_test, X_train, Y_train))

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_48 (Dense)             (None, 1)                 7133      
_________________________________________________________________
dense_49 (Dense)             (None, 1)                 2         
Total params: 7,135
Trainable params: 7,135
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
3344/3344 [==============================] - 1s 199us/sample - loss: 0.5940 - acc: 0.8615 - mean_squared_error: 0.2014
Epoch 2/100
3344/3344 [==============================] - 0s 146us/sample - loss: 0.4290 - acc: 0.8657 - mean_squared_error: 0.1274
Epoch 3/100
3344/3344 [==============================] - 1s 175us/sample - loss: 0.3331 - acc: 0.8657 - mean_squared_error: 0.0921
Epoch 4/100
3344/3344 [==============================] - 1s 160us/sample - loss: 0.2761 - acc: 0.8657 - mean_squared_err

3344/3344 [==============================] - 1s 156us/sample - loss: 0.0120 - acc: 0.9994 - mean_squared_error: 0.0010
Epoch 59/100
3344/3344 [==============================] - 1s 151us/sample - loss: 0.0114 - acc: 0.9994 - mean_squared_error: 9.7486e-04
Epoch 60/100
3344/3344 [==============================] - 1s 150us/sample - loss: 0.0109 - acc: 0.9994 - mean_squared_error: 9.0925e-04
Epoch 61/100
3344/3344 [==============================] - 1s 153us/sample - loss: 0.0104 - acc: 0.9994 - mean_squared_error: 8.3477e-04
Epoch 62/100
3344/3344 [==============================] - 1s 153us/sample - loss: 0.0099 - acc: 0.9994 - mean_squared_error: 7.7715e-04
Epoch 63/100
3344/3344 [==============================] - 0s 147us/sample - loss: 0.0095 - acc: 0.9994 - mean_squared_error: 7.2031e-04
Epoch 64/100
3344/3344 [==============================] - 0s 146us/sample - loss: 0.0090 - acc: 0.9994 - mean_squared_error: 6.6794e-04
Epoch 65/100
3344/3344 [==============================] - 1s 150u

3344/3344 [==============================] - 0s 143us/sample - loss: 0.1404 - acc: 0.8672 - mean_squared_error: 0.0426
Epoch 14/100
3344/3344 [==============================] - 0s 143us/sample - loss: 0.1328 - acc: 0.8672 - mean_squared_error: 0.0402
Epoch 15/100
3344/3344 [==============================] - 0s 141us/sample - loss: 0.1257 - acc: 0.9225 - mean_squared_error: 0.0379
Epoch 16/100
3344/3344 [==============================] - 0s 149us/sample - loss: 0.1189 - acc: 0.9871 - mean_squared_error: 0.0355
Epoch 17/100
3344/3344 [==============================] - 0s 149us/sample - loss: 0.1125 - acc: 0.9892 - mean_squared_error: 0.0333
Epoch 18/100
3344/3344 [==============================] - 0s 149us/sample - loss: 0.1063 - acc: 0.9910 - mean_squared_error: 0.0311
Epoch 19/100
3344/3344 [==============================] - 0s 143us/sample - loss: 0.1005 - acc: 0.9919 - mean_squared_error: 0.0289
Epoch 20/100
3344/3344 [==============================] - 0s 144us/sample - loss: 0.0950 

3344/3344 [==============================] - 0s 144us/sample - loss: 0.0061 - acc: 0.9994 - mean_squared_error: 3.6902e-04
Epoch 75/100
3344/3344 [==============================] - 0s 139us/sample - loss: 0.0058 - acc: 0.9997 - mean_squared_error: 3.4414e-04
Epoch 76/100
3344/3344 [==============================] - 0s 143us/sample - loss: 0.0056 - acc: 0.9997 - mean_squared_error: 3.2093e-04
Epoch 77/100
3344/3344 [==============================] - 0s 142us/sample - loss: 0.0053 - acc: 0.9997 - mean_squared_error: 3.0064e-04
Epoch 78/100
3344/3344 [==============================] - 0s 140us/sample - loss: 0.0051 - acc: 0.9997 - mean_squared_error: 2.8668e-04
Epoch 79/100
3344/3344 [==============================] - 0s 140us/sample - loss: 0.0049 - acc: 1.0000 - mean_squared_error: 2.6386e-04
Epoch 80/100
3344/3344 [==============================] - 0s 140us/sample - loss: 0.0047 - acc: 1.0000 - mean_squared_error: 2.4718e-04
Epoch 81/100
3344/3344 [==============================] - 0s 

3344/3344 [==============================] - 1s 154us/sample - loss: 0.0512 - acc: 0.9979 - mean_squared_error: 0.0108
Epoch 30/100
3344/3344 [==============================] - 0s 149us/sample - loss: 0.0483 - acc: 0.9979 - mean_squared_error: 0.0099
Epoch 31/100
3344/3344 [==============================] - 0s 148us/sample - loss: 0.0457 - acc: 0.9979 - mean_squared_error: 0.0090
Epoch 32/100
3344/3344 [==============================] - 0s 148us/sample - loss: 0.0432 - acc: 0.9979 - mean_squared_error: 0.0083
Epoch 33/100
3344/3344 [==============================] - 1s 152us/sample - loss: 0.0408 - acc: 0.9982 - mean_squared_error: 0.0075
Epoch 34/100
3344/3344 [==============================] - 1s 154us/sample - loss: 0.0386 - acc: 0.9988 - mean_squared_error: 0.0069
Epoch 35/100
3344/3344 [==============================] - 1s 153us/sample - loss: 0.0366 - acc: 0.9988 - mean_squared_error: 0.0063
Epoch 36/100
3344/3344 [==============================] - 1s 154us/sample - loss: 0.0346 

3344/3344 [==============================] - 0s 149us/sample - loss: 0.0026 - acc: 1.0000 - mean_squared_error: 7.4137e-05
Epoch 91/100
3344/3344 [==============================] - 1s 150us/sample - loss: 0.0024 - acc: 1.0000 - mean_squared_error: 6.7997e-05
Epoch 92/100
3344/3344 [==============================] - 0s 143us/sample - loss: 0.0023 - acc: 1.0000 - mean_squared_error: 6.2658e-05
Epoch 93/100
3344/3344 [==============================] - 0s 138us/sample - loss: 0.0022 - acc: 1.0000 - mean_squared_error: 5.7458e-05
Epoch 94/100
3344/3344 [==============================] - 0s 140us/sample - loss: 0.0021 - acc: 1.0000 - mean_squared_error: 5.3628e-05
Epoch 95/100
3344/3344 [==============================] - 0s 141us/sample - loss: 0.0020 - acc: 1.0000 - mean_squared_error: 4.8487e-05
Epoch 96/100
3344/3344 [==============================] - 0s 141us/sample - loss: 0.0020 - acc: 1.0000 - mean_squared_error: 4.5659e-05
Epoch 97/100
3344/3344 [==============================] - 0s 

3345/3345 [==============================] - 1s 194us/sample - loss: 0.0222 - acc: 0.9994 - mean_squared_error: 0.0028
Epoch 47/100
3345/3345 [==============================] - 1s 175us/sample - loss: 0.0211 - acc: 0.9994 - mean_squared_error: 0.0026
Epoch 48/100
3345/3345 [==============================] - 1s 183us/sample - loss: 0.0201 - acc: 0.9994 - mean_squared_error: 0.0024
Epoch 49/100
3345/3345 [==============================] - 1s 181us/sample - loss: 0.0191 - acc: 0.9994 - mean_squared_error: 0.0022
Epoch 50/100
3345/3345 [==============================] - 1s 182us/sample - loss: 0.0181 - acc: 0.9994 - mean_squared_error: 0.0020
Epoch 51/100
3345/3345 [==============================] - 1s 181us/sample - loss: 0.0172 - acc: 0.9994 - mean_squared_error: 0.0018
Epoch 52/100
3345/3345 [==============================] - 1s 209us/sample - loss: 0.0164 - acc: 0.9994 - mean_squared_error: 0.0017
Epoch 53/100
3345/3345 [==============================] - 1s 210us/sample - loss: 0.0156 

In [28]:
print("")
print("MSE Validación cruzada: ", sum(crossval)/len(crossval))    


MSE Validación cruzada:  0.021964048501104116
